# Imports

In [79]:
import pandas as pd
from collections import defaultdict
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Loading data and some stats

In [80]:
movies_data = pd.read_csv("movies_metadata.csv", usecols=['id', 'overview', 'original_language', 'revenue', 'budget', 'genres'])

box_office_data = pd.read_csv("box_office_collections.csv").drop(columns=['Movie Name', 'imdbId'])

/var/folders/9k/3s1sdkq137b5wwqwgyh27wj80000gn/T/ipykernel_23730/3957271433.py:3: DtypeWarning: Columns (13,69,92,96,100,104,105,106,107,110,113,115,120,121,122,123,124,126,127,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  box_office_data = pd.read_csv("box_office_collections.csv").drop(columns=['Movie Name', 'imdbId'])


In [81]:
# Number of rows with missing id
print(movies_data['id'].isna().sum())
print(box_office_data['id'].isna().sum())
print("NaN revenues: ", movies_data['revenue'].isna().sum())
print("0 revenues: ", movies_data['revenue'].eq(0).sum())

print("Revenue range: [", movies_data['revenue'].min(), ", ", movies_data['revenue'].max(), "]")

# Create bins for revenue, first bin with NaN, second with 0s and the rest with equal width
step = movies_data['revenue'].max() / 10
bins = [-0.1, 0.1] + [step * i for i in range(1, 11)]
revenue_bins = pd.cut(movies_data['revenue'], bins)
print("Revenue bins: ", revenue_bins.value_counts())

0
0
NaN revenues:  0
0 revenues:  25061
Revenue range: [ 0.0 ,  8425441842.34856 ]
Revenue bins:  revenue
(-0.1, 0.1]                         25061
(0.1, 842544184.235]                20127
(842544184.235, 1685088368.47]        147
(1685088368.47, 2527632552.705]        14
(2527632552.705, 3370176736.939]        6
(3370176736.939, 4212720921.174]        3
(4212720921.174, 5055265105.409]        1
(5897809289.644, 6740353473.879]        1
(7582897658.114, 8425441842.349]        1
(5055265105.409, 5897809289.644]        0
(6740353473.879, 7582897658.114]        0
Name: count, dtype: int64


In [82]:
# Budget bins
print("NaN budgets: ", movies_data['budget'].isna().sum())
numerical_budgets = movies_data['budget']

print("Numerical budgets: ", numerical_budgets.shape[0])
print("Budget range: [", numerical_budgets.min(), ", ", numerical_budgets.max(), "]")

step = numerical_budgets.max() / 10
bins = [-0.1, 0.1] + [step * i for i in range(1, 11)]
budget_bins = pd.cut(numerical_budgets, bins)
print("Budget bins: ", budget_bins.value_counts())

NaN budgets:  0
Numerical budgets:  45361
Budget range: [ 0 ,  380000000 ]
Budget bins:  budget
(-0.1, 0.1]                   36476
(0.1, 38000000.0]              7374
(38000000.0, 76000000.0]        928
(76000000.0, 114000000.0]       298
(114000000.0, 152000000.0]      163
(152000000.0, 190000000.0]       64
(190000000.0, 228000000.0]       35
(228000000.0, 266000000.0]       19
(266000000.0, 304000000.0]        3
(342000000.0, 380000000.0]        1
(304000000.0, 342000000.0]        0
Name: count, dtype: int64


# Data cleaning

## IDs

In [83]:
print("Shape before ", movies_data.shape)
# movies_data = movies_data[movies_data['id'].str.isnumeric()]
# movies_data['id'] = movies_data['id'].astype(int)
print("Shape after dropping rows with invalid ids: ", movies_data.shape)

Shape before  (45361, 6)
Shape after dropping rows with invalid ids:  (45361, 6)


## Budget

In [84]:
# Add columns budget_unknown
movies_data['budget'] = pd.to_numeric(movies_data['budget'], errors='coerce', downcast='float')
movies_data['budget_unknown'] = movies_data['budget'].apply(lambda x: 1 if x == 0.0 else 0)
movies_data['budget_100M'] = movies_data['budget'] / 1e8
print("Number of non-zero budgets: ", movies_data['budget'].ne(0).sum())
print("Number of unknown budgets: ", movies_data['budget_unknown'].sum())

Number of non-zero budgets:  8885
Number of unknown budgets:  36476


In [85]:
# Stats for budget_100M
print("Range of budget_100M: [", movies_data['budget_100M'].min(), ", ", movies_data['budget_100M'].max(), "]")
print("Mean: ", movies_data['budget_100M'].mean())
print("Median: ", movies_data['budget_100M'].median())
print("Std: ", movies_data['budget_100M'].std())

Range of budget_100M: [ 0.0 ,  3.8 ]
Mean:  0.042339819796080334
Median:  0.0
Std:  0.17442577736732917


## Revenue

In [86]:
# print("Shape before ", movies_data.shape)
# movies_data.dropna(subset=['revenue'], inplace=True)
# movies_data['revenue_100M'] = movies_data['revenue'] / 1e8
# print("Shape after dropping rows with NaN revenues and appending revenue_100M column: ", movies_data.shape)

In [87]:
# # Stats for revenue_100M
# print("Range of revenue_100M: [", movies_data['revenue_100M'].min(), ", ", movies_data['revenue_100M'].max(), "]")
# print("Mean: ", movies_data['revenue_100M'].mean())
# print("Median: ", movies_data['revenue_100M'].median())
# print("Std: ", movies_data['revenue_100M'].std())

## Overview

In [88]:
movies_data['overview'] = movies_data['overview'].fillna('')

## Country-wise revenues

In [89]:
box_office_data.head()

,id,Argentina,Aruba,Australia,Austria,Bahrain,Belgium,Bolivia,Brazil,Bulgaria,...,Guatemala,Netherlands Antilles,North Macedonia,South Africa/Nigeria,Switzerland (French/Italian),E/W Africa,Laos,Bosnia,Soviet Union,Malta
0,133185,NaN,NaN,"$1,203,589","$482,703",NaN,"$2,444,798","$4,083",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,133195,"$360,654",NaN,"$1,750,077","$479,263","$335,561","$596,815","$68,107","$1,051,166","$104,255",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,133199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,133219,"$654,232",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,133225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$1,667",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
print("Shape before ", box_office_data.shape)
box_office_data[box_office_data.columns[1:]] = box_office_data[box_office_data.columns[1:]].replace('[\$,]', '', regex=True).astype(float)
# Append revenue_ to all column names except id
new_cols = [(col, 'revenue_'+col+'_M') for col in box_office_data.columns[1:]]
box_office_data.rename(columns=dict(new_cols), inplace=True)
# Convert revenue to millions
country_cols = box_office_data.columns[1:]
box_office_data[country_cols] = box_office_data[country_cols] / 1e6
print("Shape after ", box_office_data.shape)

Shape before  (10076, 130)
Shape after  (10076, 130)


## Movies metadata after initial cleanup

In [91]:
print("Shape: ", movies_data.shape)
print("dtypes:", movies_data.dtypes)


Shape:  (45361, 8)
dtypes: budget               float64
genres                object
id                     int64
original_language     object
overview              object
revenue              float64
budget_unknown         int64
budget_100M          float64
dtype: object


In [92]:
movies_data.head()

,budget,genres,id,original_language,overview,revenue,budget_unknown,budget_100M
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782,en,"Orson Welles' unfinished masterpiece, restored...",0.000000e+00,0,0.12
1,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",38700,en,The continuing adventures of Miami detectives ...,4.970745e+08,1,0.00
2,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",332283,en,The love affair between poet Percy Shelley and...,2.443502e+06,1,0.00
3,0.0,"[{'id': 18, 'name': 'Drama'}]",412059,en,"In forgotten towns along the American border, ...",0.000000e+00,1,0.00
4,18000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",302349,en,"Twenty years after the events of Iron Sky, the...",4.672327e+05,0,0.18


## Country-wise revenue after initial cleanup

In [93]:
print("Shape: ", box_office_data.shape)
print("dtypes:", box_office_data.dtypes)

Shape:  (10076, 130)
dtypes: id                          int64
revenue_Argentina_M       float64
revenue_Aruba_M           float64
revenue_Australia_M       float64
revenue_Austria_M         float64
                           ...   
revenue_E/W Africa_M      float64
revenue_Laos_M            float64
revenue_Bosnia_M          float64
revenue_Soviet Union_M    float64
revenue_Malta_M           float64
Length: 130, dtype: object


In [94]:
box_office_data.head()

,id,revenue_Argentina_M,revenue_Aruba_M,revenue_Australia_M,revenue_Austria_M,revenue_Bahrain_M,revenue_Belgium_M,revenue_Bolivia_M,revenue_Brazil_M,revenue_Bulgaria_M,...,revenue_Guatemala_M,revenue_Netherlands Antilles_M,revenue_North Macedonia_M,revenue_South Africa/Nigeria_M,revenue_Switzerland (French/Italian)_M,revenue_E/W Africa_M,revenue_Laos_M,revenue_Bosnia_M,revenue_Soviet Union_M,revenue_Malta_M
0,133185,NaN,NaN,1.203589,0.482703,NaN,2.444798,0.004083,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,133195,0.360654,NaN,1.750077,0.479263,0.335561,0.596815,0.068107,1.051166,0.104255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,133199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,133219,0.654232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,133225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Join with country-wise revenue data

In [95]:
merged_data = pd.merge(movies_data, box_office_data, how='left', on='id')
print("Shape after merge: ", merged_data.shape)
merged_data.head()

Shape after merge:  (45361, 137)


,budget,genres,id,original_language,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Aruba_M,...,revenue_Guatemala_M,revenue_Netherlands Antilles_M,revenue_North Macedonia_M,revenue_South Africa/Nigeria_M,revenue_Switzerland (French/Italian)_M,revenue_E/W Africa_M,revenue_Laos_M,revenue_Bosnia_M,revenue_Soviet Union_M,revenue_Malta_M
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782,en,"Orson Welles' unfinished masterpiece, restored...",0.000000e+00,0,0.12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",38700,en,The continuing adventures of Miami detectives ...,4.970745e+08,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",332283,en,The love affair between poet Percy Shelley and...,2.443502e+06,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,"[{'id': 18, 'name': 'Drama'}]",412059,en,"In forgotten towns along the American border, ...",0.000000e+00,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",302349,en,"Twenty years after the events of Iron Sky, the...",4.672327e+05,0,0.18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Stats for country-wise revenue columns with mostly missing values

In [96]:
# For each column, count the number of rows with NaN values. Also calculate the number of rows with NaN values in any and all columns.

print("Rows with any Na values: ", merged_data.isna().any(axis=1).sum())
print("Rows with all Na values: ", merged_data.isna().all(axis=1).sum())

x = merged_data.isna().sum()

# Print cols with only a few non-null values
print("Fewer than 10 non-null values ", len(x[x > merged_data.shape[0] - 10]))
print("Fewer than 50 non-null values ", len(x[x > merged_data.shape[0] - 50]))
print("Fewer than 100 non-null values ", len(x[x > merged_data.shape[0] - 100]))
print("Fewer than 500 non-null values ", len(x[x > merged_data.shape[0] - 500]))
print("Fewer than 1000 non-null values ", len(x[x > merged_data.shape[0] - 1000]))

non_na = merged_data.notna().sum()
print("Columns with more than 500 non-null values ", len(non_na[non_na > 500]))
print(non_na[non_na > 500])

print("Count of Na values in each column:")
pd.set_option("display.max_rows", 200)
print(x)
pd.reset_option("display.max_rows")

Rows with any Na values:  45361
Rows with all Na values:  0
Fewer than 10 non-null values  37
Fewer than 50 non-null values  54
Fewer than 100 non-null values  67
Fewer than 500 non-null values  112
Fewer than 1000 non-null values  129
Columns with more than 500 non-null values  25
budget                      45361
genres                      45361
id                          45361
original_language           45350
overview                    45361
revenue                     45361
budget_unknown              45361
budget_100M                 45361
revenue_Argentina_M           540
revenue_Australia_M           820
revenue_Austria_M             549
revenue_Belgium_M             505
revenue_Domestic_M            693
revenue_France_M              778
revenue_Germany_M             695
revenue_Italy_M               676
revenue_Mexico_M              659
revenue_Netherlands_M         516
revenue_New Zealand_M         734
revenue_Portugal_M            550
revenue_Russia/CIS_M          570
rev

### Remove revenue columns with less than 500 non-null values

In [97]:
# Pick countries with more than 500 country-wise revenue rows
countries = x[x > merged_data.shape[0] - 500].index.tolist()
merged_data.drop(columns=countries, inplace=True)
print("Shape after dropping columns with fewer than 1000 non-null values ", merged_data.shape)
merged_data.head()

Shape after dropping columns with fewer than 1000 non-null values  (45361, 25)


,budget,genres,id,original_language,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,...,revenue_Italy_M,revenue_Mexico_M,revenue_Netherlands_M,revenue_New Zealand_M,revenue_Portugal_M,revenue_Russia/CIS_M,revenue_South Korea_M,revenue_Spain_M,revenue_Taiwan_M,revenue_United Kingdom_M
0,12000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",299782,en,"Orson Welles' unfinished masterpiece, restored...",0.000000e+00,0,0.12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",38700,en,The continuing adventures of Miami detectives ...,4.970745e+08,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",332283,en,The love affair between poet Percy Shelley and...,2.443502e+06,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,"[{'id': 18, 'name': 'Drama'}]",412059,en,"In forgotten towns along the American border, ...",0.000000e+00,1,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",302349,en,"Twenty years after the events of Iron Sky, the...",4.672327e+05,0,0.18,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# Drop rows with NaN revenue in all 17 chosen countries
country_cols = merged_data.columns[merged_data.columns.str.contains('revenue_')]
merged_data.dropna(subset=country_cols, how='all', inplace=True)
print("Shape after dropping rows with NaN revenue in all 17 chosen countries ", merged_data.shape)

Shape after dropping rows with NaN revenue in all 17 chosen countries  (1817, 25)


## Multi-hot encode Production Countries (skipped for now)

In [99]:
# print("All production_countries values: ", merged_data['production_countries'].unique())

# # Read the array inside each production_countries cell as a list, and convert it into a list of country_ids, where country_id is the index in dictionary built from all unique countries encountered in the list in each cell of production_countries column
# def get_country_isos(production_country):
#     country_isos = []
#     for country in ast.literal_eval(production_country):
#         country_isos.append(country['iso_3166_1'])
#     return country_isos

# merged_data['production_countries_isos'] = merged_data['production_countries'].apply(get_country_isos)

# # multi-hot encode the production_countries column
# mlb = MultiLabelBinarizer()
# mlb.fit(merged_data['production_countries_isos'])
# print("Total number of classes: ", len(mlb.classes_))
# print("Classes: ", mlb.classes_)

# multi_hot_encoded_countries = mlb.transform(merged_data['production_countries_isos'])
# # Create a dataframe with the multi-hot encoded columns, where column names are 'production_country_' + mlb.classes_
# multi_hot_encoded_countries_df = pd.DataFrame(multi_hot_encoded_countries, columns=['production_country_' + country for country in mlb.classes_])

# merged_data = pd.concat([merged_data, multi_hot_encoded_countries_df], axis=1)
# merged_data.drop(columns=['production_countries', 'production_countries_isos'], inplace=True)
# merged_data.head()

## One-hot encode languages

In [100]:
print("All original_language values: ", merged_data['original_language'].unique())

# Convert original_language to one-hot encoding, including NaN values
merged_data = pd.get_dummies(merged_data, columns=['original_language'], dummy_na=True)
merged_data.head()

All original_language values:  ['en' 'sv' 'ru' 'hi' 'it' 'de' 'es' 'fr' 'cn' 'pt' 'ja' 'da' 'nl' 'ko'
 'fi' 'zh' 'ta' 'no' 'ca' 'th' 'pl' 'kk' 'ro' 'cs' 'et' 'sr' 'bs' 'te'
 'fa' 'zu' 'sl' 'xx' 'tr' 'cy' 'id' 'tl' 'hu' 'bn' 'el' 'uk']


,budget,genres,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,...,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_xx,original_language_zh,original_language_zu,original_language_nan
3831,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,...,False,False,False,False,False,False,False,False,False,False
4152,140000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4515,0.0,"[{'id': 99, 'name': 'Documentary'}]",173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
5133,175000.0,"[{'id': 99, 'name': 'Documentary'}]",173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
5445,5000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


## Multi-hot encode genres

In [101]:
merged_data['genres'].head()

3831    [{'id': 10749, 'name': 'Romance'}, {'id': 18, ...
4152    [{'id': 12, 'name': 'Adventure'}, {'id': 18, '...
4515                  [{'id': 99, 'name': 'Documentary'}]
5133                  [{'id': 99, 'name': 'Documentary'}]
5445    [{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...
Name: genres, dtype: object

In [102]:
# Read the array inside each genres cell as a list
def get_genre_list(genres):
    genre_list = []
    for genre in ast.literal_eval(genres):
        genre_list.append(genre['name'])
    return genre_list

merged_data['genres_list'] = merged_data['genres'].apply(get_genre_list)
merged_data.head()

,budget,genres,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,...,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_uk,original_language_xx,original_language_zh,original_language_zu,original_language_nan,genres_list
3831,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,...,False,False,False,False,False,False,False,False,False,"[Romance, Drama]"
4152,140000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,"[Adventure, Drama, Action]"
4515,0.0,"[{'id': 99, 'name': 'Documentary'}]",173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,[Documentary]
5133,175000.0,"[{'id': 99, 'name': 'Documentary'}]",173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,[Documentary]
5445,5000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,"[Horror, Thriller]"


In [103]:
# multi-hot encode the genres column
genres_mlb = MultiLabelBinarizer()
genres_mlb.fit(merged_data['genres_list'])
print("Total number of genres: ", len(genres_mlb.classes_))
print(genres_mlb.classes_)

multi_hot_encoded_genres = genres_mlb.transform(merged_data['genres_list'])

# Create a dataframe with the multi-hot encoded columns, where column names are 'genre_' + mlb.classes_
multi_hot_encoded_genres_df = pd.DataFrame(multi_hot_encoded_genres, columns=['genre_' + genre for genre in genres_mlb.classes_])

# Append the multi-hot encoded columns to the dataframe and drop the original genres column
merged_data.reset_index(drop=True, inplace=True)
multi_hot_encoded_genres_df.reset_index(drop=True, inplace=True)
merged_data = pd.concat([merged_data, multi_hot_encoded_genres_df], axis=1)
merged_data.drop(columns=['genres', 'genres_list'], inplace=True)

print("Shape after multi-hot encoding genres: ", merged_data.shape)
merged_data.head()

Total number of genres:  20
['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary' 'Drama'
 'Family' 'Fantasy' 'Foreign' 'History' 'Horror' 'Music' 'Mystery'
 'Romance' 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
Shape after multi-hot encoding genres:  (1817, 84)


,budget,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,revenue_Belgium_M,...,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,0.0,86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,0.30037,...,0,0,0,0,1,0,0,0,0,0
1,140000000.0,147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0.0,173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,175000.0,173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5000000.0,155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,1,0,0


In [104]:
# Rows with any NaN values in columns other than country wise revenue columns starting with revenue
non_revenue_cols = [col for col in merged_data.columns if not col.startswith('revenue_')]
print("Rows with any Na values except in revenue cols: ", merged_data[non_revenue_cols].isna().any(axis=1).sum())

Rows with any Na values except in revenue cols:  0


# Training model for 'revenue' column prediction

In [105]:
# Final data
data = merged_data
data.head()

,budget,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,revenue_Belgium_M,...,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,0.0,86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,0.30037,...,0,0,0,0,1,0,0,0,0,0
1,140000000.0,147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0.0,173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,175000.0,173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5000000.0,155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,1,0,0


# Join with cast and crew data

In [106]:
credits_data = pd.read_csv("credits.csv")
print("Shape of credits data: ", credits_data.shape)

Shape of credits data:  (45476, 3)


In [107]:
credits_data.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [108]:
# Join credits_data with data
data = pd.merge(data, credits_data, how='left', on='id')

In [109]:
data.head()

,budget,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,revenue_Belgium_M,...,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western,cast,crew
0,0.0,86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,0.30037,...,0,0,1,0,0,0,0,0,"[{'cast_id': 3, 'character': 'Rick', 'credit_i...","[{'credit_id': '572dbcf5c3a3680fe5002819', 'de..."
1,140000000.0,147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,"[{'cast_id': 1, 'character': 'Moses', 'credit_...","[{'credit_id': '52fe4ba69251416c75109a81', 'de..."
2,0.0,173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,"[{'cast_id': 16, 'character': 'Himself', 'cred...","[{'credit_id': '52fe4d30c3a36847f82539d5', 'de..."
3,175000.0,173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,"[{'cast_id': 2, 'character': 'Himself', 'credi...","[{'credit_id': '52fe4d2dc3a36847f8252e37', 'de..."
4,5000000.0,155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,"[{'cast_id': 3, 'character': 'Elliot', 'credit...","[{'credit_id': '53b3c4f80e0a26598300bddb', 'de..."


In [110]:
# Get top 1000 cast members' id and name for the dataset
top_cast = defaultdict(int)
def update_cast_count(cast_list):
    for cast in ast.literal_eval(cast_list):
        top_cast[(cast['id'], cast['name'])] += 1

data['cast'].apply(update_cast_count)

# Sort the cast by count and pick top 1000
top_cast = sorted(top_cast.items(), key=lambda x: x[1], reverse=True)[:1000]
top_1000_cast_ids = [cast[0][0] for cast in top_cast]
top_1000_cast_names = [cast[0][1] for cast in top_cast]
print("Top 50 cast members: ", top_1000_cast_names[:50])


Top 50 cast members:  ['John Carradine', 'Matt Damon', 'Gene Hackman', 'John Hurt', 'Ving Rhames', 'Gérard Depardieu', 'Tom Hanks', 'Julia Roberts', 'Donald Pleasence', 'Christian Bale', 'Robert De Niro', 'Drew Barrymore', 'Sylvester Stallone', 'Harvey Keitel', 'Jean-Claude Brialy', 'Mel Gibson', 'James Stewart', 'Bess Flowers', 'Danny Trejo', 'Donald Sutherland', 'Michael Caine', 'Brad Pitt', 'Forest Whitaker', 'Sam Rockwell', 'Christopher Lee', 'Eric Roberts', 'Bruce Willis', 'Harry Dean Stanton', 'James Woods', 'M. Emmet Walsh', 'Ward Bond', 'Armin Mueller-Stahl', 'Michael Cera', 'Tony Shalhoub', 'Rutger Hauer', 'Samuel L. Jackson', 'Richard Kind', 'Heather Graham', 'William H. Macy', 'Liev Schreiber', 'Burt Reynolds', 'Patrick Stewart', 'John Goodman', 'Sharon Stone', 'Morgan Freeman', 'Malcolm McDowell', 'John Cusack', 'Bill Nighy', 'Charles Durning', 'Anthony Hopkins']


In [111]:
# Get top 500 crew members' id and name for the dataset
top_crew = defaultdict(int)
def update_crew_count(crew_list):
    for crew in ast.literal_eval(crew_list):
        top_crew[(crew['id'], crew['name'])] += 1

data['crew'].apply(update_crew_count)

# Sort the crew by count and pick top 500
top_crew = sorted(top_crew.items(), key=lambda x: x[1], reverse=True)[:500]
top_100_crew_ids = [crew[0][0] for crew in top_crew]
top_100_crew_names = [crew[0][1] for crew in top_crew]
print("Top crew members: ", top_100_crew_names)

Top crew members:  ['David Lynch', 'Charlie Chaplin', 'Avy Kaufman', 'Alfred Hitchcock', 'Cedric Gibbons', 'Jim Jarmusch', 'James Newton Howard', 'Steven Spielberg', 'Mario Bava', 'Bob Weinstein', 'Harvey Weinstein', 'Clint Eastwood', 'Werner Herzog', 'Peter Greenaway', 'Francis Ford Coppola', 'Jerry Goldsmith', 'Howard Hawks', 'Denise Chamian', 'Judd Apatow', 'Graeme Revell', 'Frank Marshall', 'Aki Kaurismäki', 'Roland Emmerich', 'John Ford', 'Edith Head', 'Stephen King', 'Mark Isham', 'Hans Zimmer', 'James Horner', 'Pete Anthony', 'Michael Kamen', 'Rainer Werner Fassbinder', 'Alfred Newman', 'Mary Vernieu', 'Martin Scorsese', 'Ellen Lewis', 'Debra Zane', 'M. Night Shyamalan', 'Sylvester Stallone', 'Bob Badami', 'Alan Silvestri', 'Roger Corman', 'Jane Jenkins', 'Charles Band', 'Claude Chabrol', 'François Truffaut', 'Otto Preminger', 'Roberto Rossellini', 'Neal H. Moritz', 'Suzanne Smith', 'Thomas Newman', 'Anne McCarthy', 'Kerry Barden', 'Tom Tykwer', 'Lucinda Syson', 'Joel Silver', '

In [112]:
# Get ids for cast members present in top 1000 cast
def get_top_cast_ids(cast):
    top_cast_ids = []
    for cast_member in ast.literal_eval(cast):
        if cast_member['id'] in top_1000_cast_ids:
            top_cast_ids.append(cast_member['id'])
    return top_cast_ids

def get_important_crew_ids(crew):
    important_crew_ids = []
    for crew_member in ast.literal_eval(crew):
        if crew_member['id'] in top_100_crew_ids:
            important_crew_ids.append(crew_member['id'])
    return important_crew_ids

data['top_cast_ids'] = data['cast'].apply(get_top_cast_ids)
data['important_crew_ids'] = data['crew'].apply(get_important_crew_ids)
data.drop(columns=['cast', 'crew'], inplace=True)
data.head()

,budget,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,revenue_Belgium_M,...,genre_Music,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western,top_cast_ids,important_crew_ids
0,0.0,86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,0.30037,...,0,0,1,0,0,0,0,0,"[3894, 112, 524, 6197, 12647, 1956, 7486, 1111...","[6410, 4185, 5632, 30715, 30715, 1440737]"
1,140000000.0,147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,"[3894, 1241, 77335, 10205, 2282, 1125]","[8401, 2260, 1404244, 1319160, 16363, 4710]"
2,0.0,173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,[],[]
3,175000.0,173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,[],[]
4,5000000.0,155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,"[2372, 3201, 19453]",[5914]


In [113]:
# Multi-hot encode the cast and crew ids
cast_mlb = MultiLabelBinarizer()
crew_mlb = MultiLabelBinarizer()

cast_mlb.fit(data['top_cast_ids'])
print("Total number of cast and crew: ", len(cast_mlb.classes_))
multi_hot_encoded_cast_ids = cast_mlb.transform(data['top_cast_ids'])

crew_mlb.fit(data['important_crew_ids'])
print("Total number of cast and crew: ", len(crew_mlb.classes_))
multi_hot_encoded_crew_ids = crew_mlb.transform(data['important_crew_ids'])

multi_hot_encoded_cast_df = pd.DataFrame(multi_hot_encoded_cast_ids, columns=['cast_' + str(cast_id) for cast_id in cast_mlb.classes_])
multi_hot_encoded_crew_df = pd.DataFrame(multi_hot_encoded_crew_ids, columns=['crew_' + str(crew_id) for crew_id in crew_mlb.classes_])

# Append the multi-hot encoded columns to the dataframe and drop the original cast and crew column
merged_data = pd.concat([data, multi_hot_encoded_cast_df, multi_hot_encoded_crew_df], axis=1)
merged_data.drop(columns=['top_cast_ids', 'important_crew_ids'], inplace=True)

Total number of cast and crew:  1000
Total number of cast and crew:  500


In [114]:
data = merged_data
data.head()

,budget,id,overview,revenue,budget_unknown,budget_100M,revenue_Argentina_M,revenue_Australia_M,revenue_Austria_M,revenue_Belgium_M,...,crew_1417400,crew_1421720,crew_1425513,crew_1440737,crew_1455461,crew_1463785,crew_1548698,crew_1552521,crew_1552549,crew_1733142
0,0.0,86835,Rick is a screenwriter living in Los Angeles. ...,6.988718e+05,1,0.00000,0.474532,0.615837,0.445917,0.30037,...,0,0,0,1,0,0,0,0,0,0
1,140000000.0,147441,The defiant leader Moses rises up against the ...,3.313432e+08,0,1.40000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0.0,173327,From Bedrooms to Billions is a 2014 documentar...,0.000000e+00,1,0.00000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,175000.0,173165,"Starting as a passion project, this movie laun...",0.000000e+00,0,0.00175,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5000000.0,155084,"A bright but meek salesman, drowning in debt a...",1.707080e+04,0,0.05000,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [115]:
data.drop(columns=['budget', 'revenue'], inplace=True)

In [116]:
# Save train and test data
data.to_csv("country_wise_train_test.csv", index=False)

In [117]:
# Save top 1000 cast and top 500 crew name to id mappings
top_cast_df = pd.DataFrame({'id': top_1000_cast_ids, 'name': top_1000_cast_names})
top_cast_df.to_csv("top_cast_country_wise.csv", index=False)
top_crew_df = pd.DataFrame({'id': top_100_crew_ids, 'name': top_100_crew_names})
top_crew_df.to_csv("top_crew_country_wise.csv", index=False)

In [118]:
print(genres_mlb.classes_)

['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Documentary' 'Drama'
 'Family' 'Fantasy' 'Foreign' 'History' 'Horror' 'Music' 'Mystery'
 'Romance' 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']
